In [3]:
# -*- coding: utf-8 -*-
import os, random, csv, glob
import cv2
import numpy as np
import pandas as pd


In [ ]:

# ============ CONFIG ============
# Ajusta estas rutas:
QR_CROPS_DIR   = r"C:\Users\ovill\OneDrive\Escritorio\QR SSD\qr\Pruebas_QR"           
BACKGROUNDS_DIR= r"C:\Users\ovill\OneDrive\Escritorio\QR SSD\IMG"             
OUTPUT_DIR     = r"C:\Users\ovill\OneDrive\Escritorio\QR SSD\sinteticos"       

# Parámetros de síntesis
NUM_IMAGES     = 8000            # total a generar
NO_QR_RATIO    = 0.30            # % de negativos sin QR
SCALE_RANGE    = (0.10, 0.30)    # el QR ocupará 10-30% del lado menor del fondo
ROT_DEG        = (-12, 12)       # rotación aleatoria
MAX_QR_PER_IMG = 1               # 1 QR por imagen (sube si quieres varios)
BLUR_P         = 0.25            # probabilidad de blur leve en el QR
NOISE_P        = 0.25            # probabilidad de ruido leve en el QR
BRICON_P       = 0.30            # probabilidad de cambiar brillo/contraste

os.makedirs(OUTPUT_DIR, exist_ok=True)

def _list_images(root):
    exts = ("*.png", "*.jpg", "*.jpeg", "*.bmp", "*.tif", "*.tiff")
    files = []
    for e in exts:
        files.extend(glob.glob(os.path.join(root, "**", e), recursive=True))
    return sorted(files)

QR_PATHS = _list_images(QR_CROPS_DIR)
BG_PATHS = _list_images(BACKGROUNDS_DIR)
assert len(QR_PATHS) > 0, "No se encontraron recortes de QR"
assert len(BG_PATHS) > 0, "No se encontraron fondos"

# ============ UTILIDADES ============
def ensure_uint8(img):
    img = np.clip(img, 0, 255)
    return img.astype(np.uint8)

def random_blur(img):
    if random.random() < BLUR_P:
        k = random.choice([3,5])
        return cv2.GaussianBlur(img, (k,k), 0)
    return img

def random_noise(img):
    if random.random() < NOISE_P:
        noise = np.random.normal(0, 8.0, img.shape).astype(np.float32)
        return ensure_uint8(img.astype(np.float32) + noise)
    return img

def random_brightness_contrast(img):
    if random.random() < BRICON_P:
        alpha = 1.0 + random.uniform(-0.15, 0.20)  # contraste
        beta  = random.uniform(-15, 20)            # brillo
        return cv2.convertScaleAbs(img, alpha=alpha, beta=beta)
    return img

def read_img_with_alpha(path):
    img = cv2.imread(path, cv2.IMREAD_UNCHANGED)
    if img is None: 
        return None, None
    if img.ndim == 3 and img.shape[2] == 4:
        b,g,r,a = cv2.split(img)
        rgb = cv2.merge([r,g,b])
        mask = a
    else:
        # Sin alpha -> generamos máscara por umbral (QR negro sobre blanco)
        bgr = img
        rgb = cv2.cvtColor(bgr, cv2.COLOR_BGR2RGB)
        gray = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
        # Umbral adaptativo robusto
        mask = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)[1]
        mask = cv2.medianBlur(mask, 3)
    return rgb, mask

def rotate_keep_bounds(img, mask, angle_deg):
    (h, w) = img.shape[:2]
    M = cv2.getRotationMatrix2D((w/2, h/2), angle_deg, 1.0)
    cos = np.abs(M[0,0]); sin = np.abs(M[0,1])
    nW = int((h*sin) + (w*cos))
    nH = int((h*cos) + (w*sin))
    M[0,2] += (nW/2) - w/2
    M[1,2] += (nH/2) - h/2

    img_rot  = cv2.warpAffine(img,  M, (nW, nH), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_CONSTANT, borderValue=255)
    mask_rot = cv2.warpAffine(mask, M, (nW, nH), flags=cv2.INTER_NEAREST, borderMode=cv2.BORDER_CONSTANT, borderValue=0)
    return img_rot, mask_rot

def place_patch(bg, patch_rgb, patch_mask, x, y):
    """Pega patch sobre bg en (x,y) usando mask. Devuelve imagen compuesta."""
    h_bg, w_bg = bg.shape[:2]
    h_p,  w_p  = patch_rgb.shape[:2]
    roi = bg[y:y+h_p, x:x+w_p].copy()

    m = (patch_mask>0).astype(np.uint8)
    m3 = cv2.merge([m,m,m])

    comp = roi*(1-m3) + patch_rgb*m3
    out = bg.copy()
    out[y:y+h_p, x:x+w_p] = comp
    return out

# ============ GENERADOR ============
annotations = []
rng = random.Random(123)

for idx in range(NUM_IMAGES):
    # 1) Carga fondo
    bg = cv2.imread(rng.choice(BG_PATHS))
    if bg is None:
        continue
    bg = cv2.cvtColor(bg, cv2.COLOR_BGR2RGB)
    h_bg, w_bg = bg.shape[:2]
    synth = bg.copy()

    rows_for_this_img = []

    # 2) ¿negativo?
    if rng.random() < NO_QR_RATIO:
        # Dummy bbox + clase 0 (negativo)
        rows_for_this_img.append(
            {"filename": f"synth_{idx:05d}.png", "xmin":0.0,"ymin":0.0,"xmax":0.0,"ymax":0.0,"class":0}
        )
    else:
        num_qr = rng.randint(1, MAX_QR_PER_IMG)
        for _ in range(num_qr):
            # 3) Carga/transforma el QR
            qr_rgb, qr_mask = read_img_with_alpha(rng.choice(QR_PATHS))
            if qr_rgb is None: 
                continue

            # Escala relativa al lado menor del fondo
            target_side = rng.uniform(*SCALE_RANGE) * min(h_bg, w_bg)
            scale = target_side / max(qr_rgb.shape[:2])
            new_w = max(5, int(qr_rgb.shape[1]*scale))
            new_h = max(5, int(qr_rgb.shape[0]*scale))
            qr_rgb  = cv2.resize(qr_rgb,  (new_w, new_h), interpolation=cv2.INTER_AREA)
            qr_mask = cv2.resize(qr_mask, (new_w, new_h), interpolation=cv2.INTER_NEAREST)

            # Augmentations leves sobre el QR
            qr_rgb  = random_blur(qr_rgb)
            qr_rgb  = random_noise(qr_rgb)
            qr_rgb  = random_brightness_contrast(qr_rgb)

            # Rotación
            angle = rng.uniform(*ROT_DEG)
            qr_rgb, qr_mask = rotate_keep_bounds(qr_rgb, qr_mask, angle)

            h_p, w_p = qr_rgb.shape[:2]
            if h_p >= h_bg or w_p >= w_bg:
                # si quedó demasiado grande, reduce
                scale2 = 0.9*min(h_bg/h_p, w_bg/w_p)
                if scale2 <= 0: 
                    continue
                qr_rgb  = cv2.resize(qr_rgb,  (int(w_p*scale2), int(h_p*scale2)))
                qr_mask = cv2.resize(qr_mask, (int(w_p*scale2), int(h_p*scale2)), interpolation=cv2.INTER_NEAREST)
                h_p, w_p = qr_rgb.shape[:2]

            # 4) Posición aleatoria que quepa
            x1 = rng.randint(0, w_bg - w_p)
            y1 = rng.randint(0, h_bg - h_p)

            # 5) Pega el QR
            synth = place_patch(synth, qr_rgb, qr_mask, x1, y1)

            # 6) BBox a partir de la máscara (rectángulo mínimo del píxel no-cero)
            ys, xs = np.where(qr_mask > 0)
            if xs.size == 0 or ys.size == 0:
                continue
            px1, py1, px2, py2 = xs.min()+x1, ys.min()+y1, xs.max()+x1, ys.max()+y1

            # Normaliza
            rows_for_this_img.append({
                "filename": f"synth_{idx:05d}.png",
                "xmin": px1 / w_bg,
                "ymin": py1 / h_bg,
                "xmax": px2 / w_bg,
                "ymax": py2 / h_bg,
                "class": 1
            })

    # 7) Guarda imagen y registra anotaciones
    out_path = os.path.join(OUTPUT_DIR, f"synth_{idx:05d}.png")
    cv2.imwrite(out_path, cv2.cvtColor(synth, cv2.COLOR_RGB2BGR))
    annotations.extend(rows_for_this_img)

# 8) Vuelca anotaciones a CSV
csv_path = os.path.join(OUTPUT_DIR, "annotations.csv")
with open(csv_path, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=["filename","xmin","ymin","xmax","ymax","class"])
    writer.writeheader()
    for r in annotations:
        writer.writerow(r)

print(f"[✓] Generadas {NUM_IMAGES} imágenes sintéticas (incluyendo {int(NO_QR_RATIO*100)}% negativas)")
print(f"[i] Carpeta: {OUTPUT_DIR}")
print(f"[i] CSV:     {csv_path}")



[✓] Generadas 8000 imágenes sintéticas (incluyendo 30% negativas)
[i] Carpeta: C:\Users\ovill\OneDrive\Escritorio\QR SSD\sinteticos
[i] CSV:     C:\Users\ovill\OneDrive\Escritorio\QR SSD\sinteticos\annotations.csv
